# Сборка семантического графа

#### Импорт библиотек

In [1]:
import geopandas as gpd
import networkx as nx
from pathlib import Path

from sloyka import Semgraph

package_dir = str(Path.cwd().parent)

#### Инициализация класса семантического графа

In [2]:
sm = Semgraph()

#### Загрузка данных
###### Рекомендуется использовать после класса Geocoder

In [3]:
data_path = package_dir + "/sloyka/sample_data/processed/"

In [4]:
test_gdf  = gpd.read_file(data_path + "-10696.geojson")

#### Построение семантического графа

In [5]:
G = sm.build_graph(test_gdf[:len(test_gdf)//2],
                   id_column='id',
                   text_column='text',
                   text_type_column='type',
                   toponym_column='only_full_street_name_numbers',
                   toponym_name_column='initial_street',
                   toponym_type_column='Toponims',
                   post_id_column='post_id',
                   parents_stack_column='parents_stack',
                   location_column='Location',
                   geometry_column='geometry')

###### Получаем количество узлов графа

In [6]:
len(G.nodes)

###### Обновляем граф

In [7]:
G = sm.update_graph(G,
                    test_gdf[len(test_gdf)//2:],
                    id_column='id',
                    text_column='text',
                    text_type_column='type',
                    toponym_column='only_full_street_name_numbers',
                    toponym_name_column='initial_street',
                    toponym_type_column='Toponims',
                    post_id_column='post_id',
                    parents_stack_column='parents_stack',
                    location_column='Location',
                    geometry_column='geometry')

###### Проверяем что количество узлов стало больше

In [8]:
len(G.nodes)

###### Смотрим атрибуты узла с номером дома в id узла

In [9]:
G.nodes['набережная канала Грибоедова 108']

###### Добавляем в атрибуты узлов со зданиями атрибут с текстом, из которого был извлечён топоним

In [10]:
G = sm.get_house_text_id(G, test_gdf, 'id', 'text')

###### Проверяем, что атрибут добавился

In [11]:
G.nodes['набережная канала Грибоедова 108']

###### Смотрим количество связей в графе

In [12]:
len(G.edges)

###### Добавляем связи расстояния между топонимами в граф

In [13]:
G = sm.calculate_distances(G)

###### Проверяем, что в граф добавились новые связи

In [14]:
len(G.edges)

###### Загружаем слои территориально-административного деления города

In [15]:
districts = gpd.read_file(data_path + "districts.geojson")
mo = gpd.read_file(data_path + "municipalities.geojson")

###### Смотрим количество узлов в графе

In [16]:
len(G.nodes)

###### Добавляем узлы районов и муниципалитетов в граф

In [17]:
G = sm.add_city_graph(G, districts, mo, 'city_id', 'district_name', 'name', 'geometry')

###### Проверяем, что в графе появились новые узлы

In [18]:
len(G.nodes)

###### Сохранение полученного графа в формате ``graphml``

In [ ]:
# nx.write_graphml(G, "semantic_graph_example.graphml", encoding = 'utf-8')